<img width="50px" align="left" style="margin-right:20px" src="http://data.newprolab.com/public-newprolab-com/npl_logo.png"> <b>New Professions Lab</b> <br /> Специалист по большим данным

# Проект 1

# Спрогнозировать пол и возрастную категорию интернет-пользователей по логу посещения сайтов

<img width="110px" align="left" src="http://data.newprolab.com/public-newprolab-com/project01_img0.png?img">

<a href="https://github.com/newprolab/content_bigdata6"><img align="left" src="http://data.newprolab.com/public-newprolab-com/npl.svg"></a>

Одна из задач DMP-системы состоит в том, чтобы по разрозненным даннным, таким, как посещения неким пользователем сайтов, классифицировать его и присвоить ему определённую категорию: пол, возраст, интересы и так далее. В дальнейшем составляется портрет, или профиль, пользователя, на основе которого ему более таргетированно показывается реклама в интернете.

### Задача

Используя доступный набор данных о посещении страниц у одной части пользователей, сделать прогноз относительно **пола и возрастной категории** другой части пользователей. Угадывание (hit) - правильное предсказание и пола, и возрастной категории одновременно.

Мы не ограничиваем вас в выборе инструментов и методов работы с данными. Используйте любые эвристики, внешние источники, парсинг контента страниц — всё, что поможет вам выполнить задачу. Единственное ограничение — никаких ручных действий. Руками проставлять классы нельзя.

Поскольку это ваш проект, который мы наверняка захотите показать другим, уделите его оформлению достаточно времени. Мы рекомендуем сделать весь проект в этом ноутбуке. Снизу, под заданием, вы сможете описать ваше решение.

⏰ **Дедлайн: 25 апреля 2017, 23:59**

In [300]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os
import json
from urlparse import urlparse
from urllib import urlretrieve, unquote, urlopen
import urllib2
import cookielib
import requests
from newspaper import Article
import nltk
import cPickle as pickle
import html2text
import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
from scipy.stats import randint as sp_randint


from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

In [52]:
df = pd.read_csv('./gender_age_dataset.txt', sep='\t')

In [3]:
df.shape

(41138, 4)

In [4]:
pd.isnull(df).sum() > 0

gender       False
age          False
uid          False
user_json    False
dtype: bool

Как мы можем увидить в данные без пропусков

In [34]:
df.groupby('gender').size()

gender
-     5000
F    17440
M    18698
dtype: int64

In [17]:
df.groupby('age').size()

age
-         5000
18-24     4898
25-34    15457
35-44     9360
45-54     4744
>=55      1679
dtype: int64

In [36]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


In [5]:
df.iloc[0].user_json

'{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}'

In [98]:
jsonv = json.loads('{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}')
for visit in jsonv.get('visits'):
    print visit['timestamp']

1419688144068
1426666298001
1426666298000
1426661722001
1426661722000


In [5]:
visit_counter=0 
for row in range(df.shape[0]): 
    visits = json.loads(df.iloc[row].user_json)
    visit_counter=visit_counter+len(visits.get('visits'))
print visit_counter

5829507


In [53]:
def get_url(url):
    try:
        a = urlparse(unquote(url.strip()))
        if a.netloc == 'news.yandex.ru':
                if(re.search('(?<=cl4url=).+(html)', url)):
                    url = 'http://' + re.search('(?<=cl4url=).+(html)', url).group(0)
                else:
                    url = 'http://' + re.search('(?<=cl4url=).+', url).group(0)
        stripUrl = str(url).strip()
        return stripUrl
    except Exception as e_get_url_in:
        pass

In [ ]:
from tqdm import tqdm
import time
urls = []
for row in tqdm(range(df.shape[0])): 
    visits = json.loads(df.iloc[row].user_json)
    for visit in visits.get('visits'):
        if str(visit['url'].encode('utf-8')).startswith('http'):
            url = get_url(visit['url'].encode('utf-8'))
            ts = visit['timestamp']
            urls.append([df.iloc[row]['uid'], url, ts, df.iloc[row]['gender']+'_'+df.iloc[row]['age']])

In [32]:
!ls .

10_userc_visits		  mapper_v2.py	     RNN_clf_url.ipynb	urls_uid_y.pkl
gender_age_dataset_h.txt  mapper_v3.py	     top350.txt		url_topic.json
gender_age_dataset.txt	  mapper_v4.py	     uniq_url		visits.txt
lab06.ipynb		  project01.ipynb    urls_h
mapper.py		  project01v1.ipynb  urls_h.pkl
mapper_v1.py		  project_env.zip    urls.pkl


In [22]:
[url[1] for url in urls[:100]]

output = open('urls_uid_y.pkl', 'wb')
pickle.dump(urls, output, 2)
output.close()

In [54]:
input = open('urls_uid_y.pkl', 'rb')
urls = pickle.load(input)
input.close()
print urls[:10]

[['d50192e5-c44e-4ae8-ae7a-7cfe67c8b777', 'http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun', 1419688144068, 'F_18-24'], ['d50192e5-c44e-4ae8-ae7a-7cfe67c8b777', 'http://chezasite.com/htc/htc-one-m9-delay-86327.html', 1426666298001, 'F_18-24'], ['d50192e5-c44e-4ae8-ae7a-7cfe67c8b777', 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html', 1426666298000, 'F_18-24'], ['d50192e5-c44e-4ae8-ae7a-7cfe67c8b777', 'http://chezasite.com/htc/htc-one-m9-delay-86327.html', 1426661722001, 'F_18-24'], ['d50192e5-c44e-4ae8-ae7a-7cfe67c8b777', 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html', 1426661722000, 'F_18-24'], ['d502331d-621e-4721-ada2-5d30b2c3801f', 'http://sweetrading.ru/?p=900', 1419717886224, 'M_25-34'], ['d502331d-621e-4721-ada2-5d30b2c3801f', 'http://sweetrading.ru/?p=884', 1419717884437, 'M_25-34'], ['d502331d-621e-4721-ada2-5d30b2c3801f', 'http://sweet

In [55]:
df_url = pd.DataFrame(urls, columns=['uid', 'url','ts', 'target'])

In [56]:
uniq_url = pd.DataFrame(list(df_url.url.unique()),columns=['uniq_url'])

In [11]:
uniq_url.to_csv('uniq_url', index=False, header=False)

In [32]:
from multiprocessing import Pool
from tqdm import tqdm

def Pars(url):
 
    article = Article(url, language='ru')
    article.download()

    try:
        article.parse()
    except:
        return ''
    
    text = article.text
    text = ' '.join(text.split())

    return (text)


for url in list(open('uniq_url_1000', 'r'))[:2]:
    try:
        text = Pars(url.replace('\n', ''))
    except:
        text = ''
 
    try:    
        print (str(url)+'\t'+text).encode('utf-8').decode('utf-8')
    except:
        try:
            print (str(url)+'\t'+text).decode('utf-8')
        except:
            print ('')

http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun
	
http://chezasite.com/htc/htc-one-m9-delay-86327.html
	HTC пришлось отодвинуть начало продаж флагмана One M9 на пока не объявленный срок. Смартфон уже сегодня должны были отправить первым тайваньским покупателям, но им придется ждать важного обновления. По информации некоторых источников, свежее системное ПО исправит некую серьезную ошибку, вызывающую проблемы с загрузкой HTC One M9. Это подтверждено и официальным письмом от производителя, который пока не называет сроков и обещает доставить обновление на гаджеты «так скоро, как будет возможно». Покупатели, успевшие пообщаться с поддержкой HTC по данному вопросу, сообщают, что компания планирует запустить продажи One M9 в течение двух недель. В качестве минимального срока исправления ошибки производитель указывает пару дней. Данная информация касается лишь основной версии смартфона, осн

In [117]:
from bs4 import BeautifulSoup
import re
#from nltk.corpus import stopwords

def text_to_wordlist(text):
   
    #text = re.sub('[а-яА-ЯёЁa-zA-Z0-9]', ' ', text)
    words = text.lower().split(',')

    #stops = set(stopwords.words("english"))

    return words

In [43]:
import nltk.data

def text_to_sentences(text):
    text = BeautifulSoup(text).get_text()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(text.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(text_to_wordlist(raw_sentence))

    return sentences

In [44]:
from multiprocessing import Pool
from tqdm import tqdm

sentences = []

text = '''HTC пришлось отодвинуть начало продаж флагмана One M9 на пока не объявленный срок. Смартфон уже сегодня должны были отправить первым тайваньским покупателям, но им придется ждать важного обновления. По информации некоторых источников, свежее системное ПО исправит некую серьезную ошибку, вызывающую проблемы с загрузкой HTC One M9. Это подтверждено и официальным письмом от производителя, который пока не называет сроков и обещает доставить обновление на гаджеты «так скоро, как будет возможно». Покупатели, успевшие пообщаться с поддержкой HTC по данному вопросу, сообщают, что компания планирует запустить продажи One M9 в течение двух недель. В качестве минимального срока исправления ошибки производитель указывает пару дней. Данная информация касается лишь основной версии смартфона, оснащенной 32-ГБ модулем памяти. Расширенный же вариант HTC One M9, получивший носитель на 64 ГБ, выйдет в продажу не раньше конца марта. Пока неясно, повлияет ли задержка с запуском смартфона на родине компании на старт продаж в остальных странах — если проблема будет исправлена уже сейчас, в России и Европе новый флагман может появиться, как и обещалось, в конце этого или начале следующего месяца. [via Focus Taiwan]'''
sentences += text_to_sentences(text)

In [ ]:
pd.to_csv(uniq_url, index=False, sep='\t', header=False)

In [83]:
topicDict = json.load(open('url_topic.json'))

In [49]:
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    if netloc is not None: return str(netloc.encode('utf8')).strip()
    return None

In [61]:
def get_domain(url):
    try:
        a = urlparse(unquote(url.strip()))
        if a.netloc == 'news.yandex.ru':
                if(re.search('(?<=cl4url=).+(html)', url)):
                    url = 'http://' + re.search('(?<=cl4url=).+(html)', url).group(0)
                else:
                    url = 'http://' + re.search('(?<=cl4url=).+', url).group(0)
        stripUrl = str(url).strip()
        return url2domain(stripUrl)
    except Exception as e_get_url_in:
        pass

In [65]:
df_url['domain'] = df_url['url'].apply(get_domain)

In [66]:
len(df_url['domain'].unique())

116790

In [86]:
df_url['topic'] = df_url['domain'].apply(lambda x: topicDict.get(x))

In [90]:
df_url['topic'].fillna('', inplace=True)

In [214]:
df_url['domain'].fillna('', inplace=True)

In [91]:
df_url.head()

,uid,url,ts,target,domain,topic
0,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://zebra-zoya.ru/200028-chehol-organayzer-...,1419688144068,F_18-24,zebra-zoya.ru,
1,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://chezasite.com/htc/htc-one-m9-delay-8632...,1426666298001,F_18-24,chezasite.com,
2,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,1426666298000,F_18-24,sotovik.ru,Мобильные телефоны
3,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://chezasite.com/htc/htc-one-m9-delay-8632...,1426661722001,F_18-24,chezasite.com,
4,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,1426661722000,F_18-24,sotovik.ru,Мобильные телефоны


In [100]:
def topic2vec(s):
    t2v = []
    for t in s.strip().split(','):
        t2v.append(t)
    return t2v

In [177]:
group_topic = df_url.groupby('uid')['topic'].agg(lambda x: ','.join(set(x))).reset_index(True)

In [215]:
group_domain = df_url.groupby('uid')['domain'].agg(lambda x: ','.join(set(x))).reset_index(True)

In [184]:
group_topic['topic'] = group_topic['topic'].apply(lambda x: x.replace(',', '', 1) if (len(x)>0 and x[0]==',') else x)

In [218]:
group_domain.head(10)

,uid,domain
0,0000e7ca-32e6-4bef-bdca-e21c025071ff,"2do2go.ru,naturalmask.ru,retail-tech.ru,astrom..."
1,0000f3cf-6e9a-4eab-92f4-cefdad108c83,"onedivision.ru,google.com,classes.ru,zenit-pen..."
2,000381a6-0400-40f8-98c0-93a6c0852d2d,"russianfood.com,google.ru"
3,00062d89-a3ba-40dd-9a25-212c9f429133,home.webalta.ru
4,0009233e-a0c2-438e-ac36-bd3e1b7dad1e,"i-sux.com,fucked-tube.com"
5,000c67ae-e4e2-465e-9b22-ce67bbe1385c,shop.lenovo.com
6,000c737c-5cfc-4bc2-aa20-daf61068a69b,"paygate.transcredit.ru,kanjiname.ru,centreopt...."
7,000de64c-b8b0-4eed-874a-06f685e17920,"segodnya.ua,forum.cn.ua,musson.ua,numizmatik.r..."
8,000e562a-fb49-4d51-a692-787fcec76931,"aif.ru,rbwdelta.narod.ru,ebay.com,catcar.info,..."
9,000fcabd-89eb-4d5a-8ad0-c2bebded6388,dating.meta.ua


In [186]:
sentences = []
for raw_sentence in group_topic.iterrows():
        sentences.append(text_to_wordlist(raw_sentence[1][1]))
len(sentences)

41138

In [232]:
sentences_dom = []
for raw_sentence in group_domain.iterrows():
        sentences_dom.append(text_to_wordlist(raw_sentence[1][1]))
len(sentences_dom)

41138

In [187]:
%%time
# симортируем соответствующую функцию из модуля gensim, который должен быть установлен

from gensim.models.word2vec import Word2Vec 

# список параметров, которые можно менять по вашему желанию
num_features = 100  # итоговая размерность вектора каждого слова
min_word_count = 1  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 4     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 4        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

CPU times: user 2.18 s, sys: 235 ms, total: 2.41 s
Wall time: 1.42 s


In [234]:
%%time
# симортируем соответствующую функцию из модуля gensim, который должен быть установлен

from gensim.models.word2vec import Word2Vec 

# список параметров, которые можно менять по вашему желанию
num_features = 100  # итоговая размерность вектора каждого слова
min_word_count = 1  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 4     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 4        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model_dom = Word2Vec(sentences_dom, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

CPU times: user 19 s, sys: 835 ms, total: 19.8 s
Wall time: 9.25 s


In [137]:
model.init_sims(replace=True)

In [235]:
model_dom.init_sims(replace=True)

In [198]:
def topics_w2v(sent, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in sent:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [249]:
def domain_w2v(sent, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in sent:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [206]:
sen_vec = []
for n in range(len(sentences)):
    sen_vec.append(topics_w2v(sentences[n],model, 100))

In [250]:
sen_dom_vec = []
for n in tqdm(range(len(sentences_dom))):
    sen_dom_vec.append(domain_w2v(sentences_dom[n],model_dom, 100))

100%|██████████| 41138/41138 [19:05<00:00, 35.93it/s]


In [224]:
topic_w2v = group_topic.join(pd.DataFrame(sen_vec))

In [252]:
domain_w2v = group_topic.join(pd.DataFrame(sen_dom_vec))

In [253]:
domain_w2v.head()

,uid,topic,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,0000e7ca-32e6-4bef-bdca-e21c025071ff,"Афиша,Учёба,Поисковые системы",0.169153,-0.099084,-0.058247,0.062917,0.081794,-0.008729,0.064927,0.003386,...,0.097591,-0.128122,0.077316,0.227314,-0.018523,0.047173,-0.038104,0.120925,0.035157,-0.071000
1,0000f3cf-6e9a-4eab-92f4-cefdad108c83,Футбол,0.202803,-0.113804,-0.060480,0.084485,0.068101,-0.002785,0.054932,0.012815,...,0.071392,-0.117050,0.081420,0.227730,-0.008774,0.048438,-0.016666,0.132887,0.051860,-0.073539
2,000381a6-0400-40f8-98c0-93a6c0852d2d,"Поисковые системы,Кулинария",0.177037,-0.099417,-0.046265,0.071207,0.077886,-0.004668,0.055286,-0.001279,...,0.094113,-0.102721,0.097214,0.212943,-0.034638,0.027696,-0.042848,0.115672,0.052930,-0.065145
3,00062d89-a3ba-40dd-9a25-212c9f429133,,0.172354,-0.090236,-0.063934,0.063963,0.094414,-0.021780,0.072835,0.010171,...,0.109554,-0.145096,0.092490,0.213237,-0.005104,0.068343,-0.026618,0.126171,0.037817,-0.057540
4,0009233e-a0c2-438e-ac36-bd3e1b7dad1e,,0.156834,-0.092972,-0.057520,0.081465,0.088406,0.018081,0.054070,0.023347,...,0.115018,-0.109232,0.071775,0.218846,-0.030919,0.024037,-0.066697,0.101290,0.029862,-0.064084


In [257]:
del domain_w2v['topic']

In [258]:
domain_w2v.to_csv('domain_w2v', index=False)

In [327]:
topic_domain_w2v = pd.merge(topic_w2v, domain_w2v, on='uid')

In [328]:
topic_domain_w2v.head()

,uid,topic,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,...,90_y,91_y,92_y,93_y,94_y,95_y,96_y,97_y,98_y,99_y
0,0000e7ca-32e6-4bef-bdca-e21c025071ff,"Афиша,Учёба,Поисковые системы",-0.136438,0.095003,0.181394,0.186484,0.043901,-0.277346,0.024829,-0.158159,...,0.097591,-0.128122,0.077316,0.227314,-0.018523,0.047173,-0.038104,0.120925,0.035157,-0.071000
1,0000f3cf-6e9a-4eab-92f4-cefdad108c83,Футбол,-0.058444,-0.015480,0.272707,-0.081244,0.024444,-0.020156,-0.206274,0.035572,...,0.071392,-0.117050,0.081420,0.227730,-0.008774,0.048438,-0.016666,0.132887,0.051860,-0.073539
2,000381a6-0400-40f8-98c0-93a6c0852d2d,"Поисковые системы,Кулинария",-0.144344,0.048023,0.273007,0.177869,0.155977,-0.212173,0.105107,-0.055093,...,0.094113,-0.102721,0.097214,0.212943,-0.034638,0.027696,-0.042848,0.115672,0.052930,-0.065145
3,00062d89-a3ba-40dd-9a25-212c9f429133,,-0.039267,0.089302,0.152713,0.185479,0.079326,-0.017916,-0.013912,-0.042354,...,0.109554,-0.145096,0.092490,0.213237,-0.005104,0.068343,-0.026618,0.126171,0.037817,-0.057540
4,0009233e-a0c2-438e-ac36-bd3e1b7dad1e,,-0.039267,0.089302,0.152713,0.185479,0.079326,-0.017916,-0.013912,-0.042354,...,0.115018,-0.109232,0.071775,0.218846,-0.030919,0.024037,-0.066697,0.101290,0.029862,-0.064084


In [329]:
del topic_domain_w2v['topic']

In [330]:
rnn_pred_url = pd.read_csv('pred_url')

In [331]:
final_pred = pd.merge(topic_domain_w2v, rnn_pred_url, on='uid')

In [332]:
final_pred = pd.merge(df, final_pred, on='uid')

In [333]:
final_pred.head()

,gender,age,uid,user_json,0_x,1_x,2_x,3_x,4_x,5_x,...,M_25-34,F_25-34,M_35-44,F_45-54,num_url,M_>=55,M_45-54,F_35-44,F_>=55,M_18-24
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000...",-0.216731,0.331878,0.406458,0.359345,-0.000569,-0.473637,...,0.852160,1.015698e-22,8.160594e-23,8.195700e-23,5,0.099088,1.198349e-22,1.475508e-22,7.206251e-23,1.200307e-22
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=...",0.008434,0.130241,0.108345,0.104738,-0.027596,-0.086027,...,0.852352,1.064481e-22,8.543414e-23,8.639450e-23,102,0.098945,1.259816e-22,1.545152e-22,7.552368e-23,1.253123e-22
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr...",-0.092895,0.081689,0.178012,0.157137,0.127349,-0.057858,...,0.852969,1.092142e-22,8.752425e-23,8.930910e-23,44,0.098634,1.297646e-22,1.584117e-22,7.758131e-23,1.281451e-22
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r...",0.171748,-0.044650,0.097662,-0.207700,-0.109285,0.087263,...,0.852998,1.017539e-22,8.164012e-23,8.208970e-23,14,0.098688,1.199734e-22,1.478358e-22,7.219968e-23,1.200848e-22
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#...",-0.086742,0.040678,0.282966,0.038776,0.061263,-0.137502,...,0.852870,1.087173e-22,8.714346e-23,8.886159e-23,212,0.098678,1.291305e-22,1.577205e-22,7.723294e-23,1.277195e-22


In [334]:
final_pred['target'] = final_pred['gender']+'_'+final_pred['age']

In [336]:
df_uid = final_pred[final_pred['target'] == '-_-']['uid']

In [337]:
del final_pred['gender']
del final_pred['age']
del final_pred['uid']
del final_pred['user_json']

In [338]:
df_train = final_pred[~(final_pred['target'] == '-_-')]
df_test = final_pred[final_pred['target'] == '-_-']
df_y = df_train['target']
del df_train['target']
del df_test['target']

In [382]:
# build a classifier
clf = RandomForestClassifier(n_estimators=300, n_jobs=-1)

# specify parameters and distributions to sample from
tree_params= {"max_depth": sp_randint(2, 20),
              "max_features": sp_randint(20, len(df_train.columns)),
              "min_samples_split": sp_randint(1, 20),
              "min_samples_leaf": sp_randint(1, 20)}

# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(clf, param_distributions=tree_params, n_iter=n_iter_search, n_jobs=-1, cv=5)

start = time()
random_search.fit(df_train, df_y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

RandomizedSearchCV took 4565.84 seconds for 10 candidates parameter settings.


In [375]:
random_search.best_params_

{'max_depth': 9,
 'max_features': 163,
 'min_samples_leaf': 4,
 'min_samples_split': 1}

In [361]:
def split_target_gen(row):
    g,a = row.split('_')
    return g

def split_target_age(row):
    g,a = row.split('_')
    return a

In [376]:
pred_class = pd.DataFrame(zip(df_uid, random_search.predict(df_test)), columns=['uid','class'])

In [377]:
pred_class['gender'] = pred_class['class'].apply(split_target_gen)
pred_class['age'] = pred_class['class'].apply(split_target_age)
del pred_class['class']

In [381]:
random_search.best_score_

0.28463113620012176

In [378]:
pred_class.head()

,uid,gender,age
0,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,M,25-34
1,bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd,M,25-34
2,bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c,M,25-34
3,bd7c5d7a-0def-41d1-895f-fdb96c56c2d4,M,25-34
4,bd7e54a2-0215-45cb-a869-9efebf250e38,M,25-34


In [373]:
pred_class.sort('uid').to_csv( "project01_gender-age.csv", index=False, sep='\t' )

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


#### Пример выходного файла:

```uid	gender	age
123	F	18-24
456	M	25-34
789	-	-
```

### Проверка
Проверка осуществляется из [Личного кабинета](http://lk.newprolab.com/lab/project01). По файлу будет определяться доля правильно спрогнозированных пользователей (у которых правильно указаны и пол, и возрастная категория).

* В поле `part of users with predicted gender + age` - указана доля пользователей, которая была предсказана от общего числа неизвестных пользователей (пример: по 3 000 был сделан прогноз, а всего было неизвестно 5 000, чекер выдаст 0.6).

* В поле `correctly predicted users / total number of users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа всех пользователей (пример: по 3 000 был сделан прогноз, правильно было спрогнозировано 1 500, а всего было неизвестно 5 000, чекер выдаст 0.3)

* В поле `correctly predicted users / number of predicted users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа предсказанных пользователей (пример: по 3 000 был сделан прогноз, из них правильно предсказано 1 500, чекер выдаст 0.5).

**Если доля в последнем поле превысит порог 0.28, то проект будет засчитан.**

Лучшей команде, набравшей максимальный результат, мы подарим специальный приз, о котором скажем позднее.